In [42]:
'''
Created on 18 dic. 2016

@author: Tobar
'''
import os
from skimage.color import rgb2lab,rgb2grey,lab2rgb,rgb2grey
from skimage.filters import threshold_otsu
import networkx as nx
from networkx.algorithms import approximation as apxa
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline  

class TestCalidad():

        
    
    def iniciar_test(self):
        actual = os.getcwd()
        
        procesado_de_imagen=ProcesadoDeImagen()
        procesado_de_lineas=ProcesadoDeLineas
        img=procesado_de_imagen.leer_imagen(actual+"/Test/codigo/calidad/imagenesPrueba/1-350-7547 1.jpg")
        l,a,b=self.pixelRGB2LAB([255, 8, 0])        
        lab=rgb2lab(img)
        distance=abs(lab - [l,a,b]).mean(axis=2)
        im=self.binarizar(distance) 
        tru_positive_inicial,tru_negative_inicial,false_positive_inicial,false_negative_inicial = self.inicial_test(im,img)
        print(tru_positive_inicial,tru_negative_inicial)
        print(false_positive_inicial,false_negative_inicial)         
        sinRuido=procesado_de_imagen.reducir_grosor(im)
        lines=procesado_de_imagen.pro_hough(10,5,11,sinRuido) 
        G=nx.Graph()
        G=procesado_de_lineas.combina(8,4,lines,G)
        k_components = apxa.k_components(G)
        segmentosDeVerdad=procesado_de_lineas.segmentos_verdad(k_components,lines)
        
        
        pathh=actual+"/Test/codigo/calidad/imagenesPrueba/1-350-7547 1SinPintar.jpg"
        temp=actual+"/Test/codigo/calidad/imagenesPrueba/"
    
        self.guardar_y_pintar( pathh, temp, segmentosDeVerdad)
    
        img_pintada=procesado_de_imagen.leer_imagen(actual+"/Test/codigo/calidad/imagenesPrueba/calculada.jpg")

        tru_positive_compara,false_negative_compara,false_positive_compara,tru_negative_compara=self.calcula_medidas(img_pintada,im)
        print( tru_positive_compara,false_negative_compara)
        print(false_positive_compara,tru_negative_compara)
        
    def pixelRGB2LAB(self,pixel):
        r,g,b = pixel
        return rgb2lab([[[r/255,g/255,b/255]]])[0][0]

    
    def binarizar(self,distance_red):
        threshold_global_otsu = threshold_otsu(distance_red)
        imgBin = distance_red <= threshold_global_otsu
        return imgBin    
    

    
    def es_rojo(self,l):
        r,g,b=l
        if r > 130 and g <90 and b < 85:
            return True
        else:
            return False
        
    def inicial_test(self,im,img):
        tru_positive_inicial=0
        tru_negative_inicial=0
        false_positive_inicial=0
        false_negative_inicial=0
        for i,ii in zip(im,img):
            for j,jj in zip(i,ii):
                if j == True and self.es_rojo(jj):
                    tru_positive_inicial=tru_positive_inicial+1
                elif self.es_rojo(jj) and j == False:
                    false_negative_inicial=false_negative_inicial+1
                elif not self.es_rojo(jj) and j == True:
                    false_positive_inicial=false_positive_inicial+1
                elif j == False and  not self.es_rojo(jj):
                    tru_negative_inicial=tru_negative_inicial+1
        return tru_positive_inicial,tru_negative_inicial,false_positive_inicial,false_negative_inicial          
    
    
 
    
    
    def guardar_y_pintar(self, path, temp, segmentos):
        """
        Metodo que se encargara de guardar y pintar los segmentos pasados 
        en la imagen para asi poder usarla para informes.
        
        @param path: Camino donde guardar la imagen con las lineas nuevas pintadas. 
        @param tem ruta al fichero temporal donde guardar la imagen.
        @param segmentos: Segmentos a pintar en la imagen.
        """
        im = Image.open(path).convert("RGB")
        draw = ImageDraw.Draw(im)
        for i in segmentos:             
            draw.line((i[0][0], i[0][1], i[1][0], i[1][1]), fill='red', width=5)         
                                   
        im.save(temp + "/calculada.jpg", "JPEG", quality=100)
    
    def calcula_medidas(self,img_pintada,im):
        tru_positive_compara=0
        tru_negative_compara=0
        false_positive_compara=0
        false_negative_compara=0
        for i,ii in zip(img_pintada,im):
            for j,jj in zip(i,ii):
                if self.es_rojo(j) and jj == True:
                    tru_positive_compara=tru_positive_compara+1
                elif jj == True and not self.es_rojo(j):
                    false_negative_compara=false_negative_compara+1
                elif jj == False and self.es_rojo(j):
                    false_positive_compara=false_positive_compara+1
                elif not self.es_rojo(j) and  jj == False:
                    tru_negative_compara=tru_negative_compara+1
        return tru_positive_compara,false_negative_compara,false_positive_compara,tru_negative_compara


    


In [43]:
uno=TestCalidad()
uno.iniciar_test()

36939 1186609
3037 2215
34755 5221
4752 1184072


In [6]:
import math
import numpy as np
class ProcesadoDeLineas():
    """
    Clase que va a contener las funciones necesarias para poder hacer
    el procesado de las lineas.
    
    @author: Ismael Tobar Garcia
    @version: 1.0
    """
    
    @classmethod
    def combina(self, epsilon1, epsilon2, lines, g): 
        """
        Funcion que combina las lineas cercanas producidas por la Transformada probabilistica
        de hough que cumplen ciertos parametros y las anade a un grafo.

        @param epsilon1: Distancia que si superan dos rectas no une
        @param epsilon2: Angulo que si superan dos rectas no une
        @param lines: Lineas producidas al aplicar la transformada
        @param G: Grafo donde anadir los nodos que tengamso que unir
 
        @return: devolvemos el Grafo completo donde estan los nodos que hemso fusionado.
        """
        for i in range(len(lines)):
            g.add_node(i)

        for i in range(len(lines) - 1):
            for j in range(i + 1, len(lines)):
                self.comprueba(lines, i, j, epsilon1, epsilon2, g)
        return g
    
    @classmethod
    def comprueba(self, lines, i, j, epsilon1, epsilon2, g):
        """
        Metodo que comprueba si hay que añadirlo al grafo o no.
        
        @param lines: Lineas producidas al aplicar la transformada.
        @param i: variable i del bucle.
        @param j: variable j del bucle.
        @param epsilon1: Distancia que si superan dos rectas no une.
        @param epsilon2: Angulo que si superan dos rectas no une.
        @param g: grafo donde estan las lineas.
        """
        distance = self.segments_distance(lines[i], lines[j])
        if distance <= epsilon1 :
                    angle = self.ang(lines[i], lines[j])
                    if angle <= epsilon2:
                        g.add_edge(i, j) 
                        
    
    
    @classmethod 
    def segments_distance(self, x, y):
        """
        Funcion que calcula la distancia entre dos segmentos que no se cruzan.        
        
        @param x: puntos x e y del punto 1
        @param y: puntos x e y del punto 2
 
        @return: min(distances): la distancia minima de todas las posibles distancias
        """
        x1, y1 = x
        x11, y11 = x1
        x12, y12 = y1
        
        x2, y2, = y
        x21, y21 = x2
        x22, y22 = y2

        if self.segments_intersect(x, y): 
                return 0
        # try each of the 4 vertices w/the other segment
        distances = []
        distances.append(self.point_segment_distance(x11, y11, x21, y21, x22, y22))
        distances.append(self.point_segment_distance(x12, y12, x21, y21, x22, y22))
        distances.append(self.point_segment_distance(x21, y21, x11, y11, x12, y12))
        distances.append(self.point_segment_distance(x22, y22, x11, y11, x12, y12))
        return min(distances)
    
    
    @classmethod
    def segments_intersect(self, x, y):
        """ 
        Funcion boleana que calcula si dos segmentos se cruzan o no.
        
        @param x: puntos x e y del punto 1.
        @param y: puntos x e y del punto 2.
        
        @return: true/false si se cruzan o no 
        """
        x1, y1 = x
        x11, y11 = x1
        x12, y12 = y1
                
        x2, y2, = y
        x21, y21 = x2
        x22, y22 = y2
        
        dx1 = x12 - x11
        dy1 = y12 - y11
        dx2 = x22 - x21
        dy2 = y22 - y21
        
        delta = dx2 * dy1 - dy2 * dx1
        if delta == 0: 
            return False  # parallel segments
        s = (dx1 * (y21 - y11) + dy1 * (x11 - x21)) / delta
        t = (dx2 * (y11 - y21) + dy2 * (x21 - x11)) / (-delta)
        return (0 <= s <= 1) and (0 <= t <= 1)
    
     
    @classmethod
    def point_segment_distance(self, px, py, x1, y1, x2, y2):
        """
        Funcion que calcula la distancia desde un punto dado a un segmento.
        @param px: Punto coordenada x 
        @param py: Punto coordenada y
        @param x1: Punto 1 coordenada x 
        @param y1: Punto 1 coordenada y
        @param x2: Punto 2 coordenada x 
        @param y2: Punto 2 coordenada y

        @return: Distancia desde el punto al segmento.
        """
        dx = x2 - x1
        dy = y2 - y1
        if dx == dy == 0:  # the segment's just a point
            return math.hypot(px - x1, py - y1)

        # Calculate the t that minimizes the distance.
        t = ((px - x1) * dx + (py - y1) * dy) / (dx * dx + dy * dy)

        # See if this represents one of the segment's
        # end points or a point in the middle.
        if t < 0:
            dx = px - x1
            dy = py - y1
        elif t > 1:
            dx = px - x2
            dy = py - y2
        else:
            near_x = x1 + t * dx
            near_y = y1 + t * dy
            dx = px - near_x
            dy = py - near_y

        return math.hypot(dx, dy)
    
    @classmethod
    def ang(self, line_a, line_b):
        """
        Funcion que dadas dos rectas calcula el angulo que forman entre ellas.
        
        @return: ang_deg: Devuelve el angulo que forman dichas rectas en grados.
        """
        # Get nicer vector form
        v_a = [line_a[0][0] - line_a[1][0], line_a[0][1] - line_a[1][1]]
        v_b = [line_b[0][0] - line_b[1][0], line_b[0][1] - line_b[1][1]]
        # Get dot prod
        dot_prod = self.dot(v_a, v_b)
        # Get magnitudes
        mag_a = self.dot(v_a, v_a) ** 0.5
        mag_b = self.dot(v_b, v_b) ** 0.5
        # Get cosine value
        cos_ = dot_prod / mag_a / mag_b
        if cos_ > 1:
            cos_ = 1
        # Get angle in radians and then convert to degrees
        angle = math.acos(cos_)
        # Basically doing angle <- angle mod 360
        ang_deg = math.degrees(angle) % 360

        if ang_deg - 180 >= 0:
            # As in if statement
            return 360 - ang_deg
        else: 
            return ang_deg
    
    
    @classmethod
    def dot(self, v_a, v_b):
        """
        Funcion que devuelve la multiplicacion de las coordenadas x de los dos puntos 
        mas la multiplicacion de las coordenadas y de los dos puntos.

        @param vA: Punto A.
        @param vB: Punto B.

        @return: la multiplicacion de las coordenadas x de los dos puntos 
               mas la multiplicacion de las coordenadas y de los dos puntos.
        """
        return v_a[0] * v_b[0] + v_a[1] * v_b[1]
    

    @classmethod
    def combina_segmentos(self, segmentos_list):
        """
        Lista con los segmentos combinados que calculamos con la teoria de grafos.
        
        @param segmentosList: lista con todos los segmentos a combinar
        Combinamos los segmentos.
        """
        xs = list(map(lambda x:[x[0][0], x[1][0]], segmentos_list))
        ys = list(map(lambda x:[x[0][1], x[1][1]], segmentos_list))
        x_max = np.max(xs)
        y_max = np.max(ys)
        x_min = np.min(xs)
        y_min = np.min(ys)
        if (x_max, y_max) in set(map(lambda x:x[0], segmentos_list)):
            return (x_max, y_max), (x_min, y_min)
        else:
            return (x_max, y_min), (x_min, y_max)
    @classmethod
    def filtra_contenidas(self,lineas,x_min,x_max,y_min,y_max,lon_min):
        lines_pintar=[]
        for i in lineas:
            existe_punto1=self.pertenece_o_no(i[0][0], i[0][1],x_min,x_max,y_min,y_max)
            existe_punto2=self.pertenece_o_no(i[1][0], i[1][1],x_min,x_max,y_min,y_max)
            if existe_punto1 and existe_punto2 and self.longitud_linea(i) > lon_min:
                lines_pintar.append(i)
        return lines_pintar
    @classmethod
    def filtra_intersec(self,lineas,cuadr,variables,lon_min):
        nuevas=[]
        x_min,x_max,y_min,y_max=variables
        for i in lineas:
            add=0
            p1,p2=i
            x1,y1=p1
            x2,y2=p2
            fl=0
            for j in cuadr:
                marc=0
                un=self.segments_intersect( i, j)
                existe_punto1=self.pertenece_o_no(i[0][0], i[0][1],x_min,x_max,y_min,y_max)
                existe_punto2=self.pertenece_o_no(i[1][0], i[1][1],x_min,x_max,y_min,y_max)
                if not existe_punto1 and fl!=1 and un:
                    tem_p1x,tem_p1y=self.seg_intersect(i,j)
                    x1=tem_p1x
                    y1=tem_p1y
                    add=1
                    marc=1
                    fl=1
                x2,y2,add=self.prueba_point2(existe_punto2, marc, un, i, j,[x2,y2,add])
                marc=0
            if add==1 and self.longitud_linea([(x1,y1),(x2,y2)]) > lon_min:
                nuevas.append([(x1,y1),(x2,y2)])
                add=0   
        return nuevas
    @classmethod
    def prueba_point2(self,existe_punto2,marc,un,i,j,dos):
        x2,y2,add=dos
        if not existe_punto2 and marc!=1 and un:
            tem_p2x,tem_p2y=self.seg_intersect(i,j)
            x2=tem_p2x
            y2=tem_p2y
            add=1
        return x2,y2,add
    @classmethod
    def pertenece_o_no(self,x,y,x_min,x_max,y_min,y_max):
        """
        En este metodo dado un punto y los 4 vertices del cuadrado comprobaremos
        si las coordenadas del punto pertenecen al area pintable.
        @param x: coordenada x del punto a comprobar.
        @param y: coordenada y del punto a comprobar.
        @param x_min: coordenada x minima del cuadrado
        @param x_max: coordenada x macima del cuadrado.
        @param y_min: coodenada y minima del cuadrado.
        @param y_max: coordenada y maxima del cuadrado.
        @return: true/false dependiendo si pertenece a esa region o no.     
        """
        if x == None and y==None:
            return False
        if (x_min< x < x_max) and  (y_min< y < y_max):
            return True
        else:
            return False
    @classmethod
    def perp( self,a ) :
        b = np.empty_like(a)
        b[0] = -a[1]
        b[1] =  a[0]
        return b
    @classmethod
    def seg_intersect(self,segment1, segment2) :
        a1, a2 = np.array(segment1)
        b1, b2 = np.array(segment2)    
        
        da = a2-a1
        db = b2-b1
        dp = a1-b1
        dap = self.perp(da)
        denom = np.dot( dap, db)
        num = np.dot( dap, dp )
        return list((num / denom.astype(float))*db + b1)
        
    @classmethod
    def longitud_linea(self, p, valor=100):
        """
        Funcion que calcula la longitud de un segmento.
        
        @param p: segmento del que calcular su distancia.
        
        @Return distancia del segmento
        """
        
        valor = int(valor)
        long_ref = int(valor)
        long_seg = (((p[1][0] - p[0][0]) ** 2) + ((p[1][1] - p[0][1]) ** 2)) ** (1 / 2)
        
        return (long_seg * valor) / long_ref
    
    @classmethod
    def segmentos_verdad(self, k_components, lines):
        """
        Funcion que dadas las lineas calculadas y las que nos marca el grafo que tenemso que unir
        devuelve lso segmentos que de verdad ha detectado el algoritmo.
        
        @param k_components: K_componentes del grafo en el que decimos que rectas tenemos que unir
        @param lines: lineas calculadas por la tranformada de hough
        
        @return: segmentos_de_verdad: lista de los segmentos que tienen.
        """
        segmentos_de_verdad = []
        for i in range(len(k_components[1])):
            segmentos = list(map(lambda x:lines[x], k_components[1][i]))           
            segmentos_de_verdad.append(self.combina_segmentos(segmentos))
        return segmentos_de_verdad
   


In [7]:
from skimage import io  # Librerias para importar las imagenes
from skimage.color import rgb2grey  # Mostrar las 3 imagenes
from skimage.color import rgb2hsv  # Mostrar las 3 imagenes
from skimage.filters import threshold_otsu
from skimage.morphology import  skeletonize 
from skimage.transform import probabilistic_hough_line
import numpy as np
import tempfile
import os,shutil
from skimage.color import rgb2lab,gray2rgb
from PIL import Image, ImageDraw
import warnings
class ProcesadoDeImagen():
    """
    Clase que contendra los metodos necesarios para poder realizar el procesado de las 
    imagenes para obtener a partir de una simple imagen las lienas que hay en ella pintadas
    en rojo.
    
    @author: Ismael Tobar Garcia
    @version: 1.0
    """
   
    @classmethod
    def leer_imagen(self, path_img):
        """
        Metodo Para leer una imagen a traves de scikit-Image (skimage)
        dandole el path de la imagen.
        @param path_img: camino hasta la imagen.
        @retrun Img: imagen leida.
        """
        self.dic=Diccionario()     
        img = io.imread(path_img.replace('\\', '/'))
        
        return gray2rgb(img)
    
    @classmethod
    def guardar_y_pintar(self, path, temp, segmentos,cuadrado):
        """
        Metodo que se encargara de guardar y pintar los segmentos pasados 
        en la imagen para asi poder usarla para informes.
        
        @param path: Camino donde guardar la imagen con las lineas nuevas pintadas. 
        @param tem ruta al fichero temporal donde guardar la imagen.
        @param segmentos: Segmentos a pintar en la imagen.
        """
        im = Image.open(path).convert("RGB")
        draw = ImageDraw.Draw(im)
        for i in segmentos:             
            draw.line((i[0][0], i[0][1], i[1][0], i[1][1]), fill='red', width=2)         
        if cuadrado[0]!="-" and cuadrado[1]!="-" and cuadrado[2]!="-" and cuadrado[3]!="-":
            draw.line((cuadrado[1]-2, cuadrado[3], cuadrado[0]+2, cuadrado[3]), fill='#020202', width=6)    
            draw.line((cuadrado[1], cuadrado[3], cuadrado[1], cuadrado[2]), fill='#020202', width=6)    
            draw.line((cuadrado[1]-2, cuadrado[2], cuadrado[0]+2, cuadrado[2]), fill='#020202', width=6)    
            draw.line((cuadrado[0], cuadrado[2], cuadrado[0], cuadrado[3]), fill='#020202', width=6) 
                           
        im.save(temp + self.dic.pintada, self.dic.jpg, quality=100)

    @classmethod
    def distancia_al_rojo(self, img,pixel):
        """
        Metodo para:
        Pasamos la imagen al espacio de color RGB y nos quedamos con el canal rojo
        Pasamos la imagen al espacio de color HSV
        Normalizamos la imagen del espacio de color HSV para utilizar que distancia 
        al rojo tenemos y poder hacer el theshold.
        
        @param img: imagen original leida anteriormente.

        @return: distance_red: distancia de cada pixel al rojo apra luego hacer el threshold.
        
        """ 
        r,g,b = pixel        
        g,v,b=rgb2lab([[[r/255,g/255,b/255]]])[0][0]
        
        lab=rgb2lab(img)
        distance_red=abs(lab - [g,v,b]).mean(axis=2)
        return distance_red


    @classmethod
    def binarizar(self, distance_red):
        """
        Metodo para binarizar la imagen para obtener una clara diferenciacion entre 
        el fondo y las lineas que queremos detectar.
        
        @param distance_red: distancia de cada pixel al rojo apra luego hacer el threshold.

        @return: imgBin: imagen binarizada.
        """
        threshold_global_otsu = threshold_otsu(distance_red)
        img_bin = distance_red <= threshold_global_otsu
        return img_bin

    
    @classmethod
    def crop_img(self, img_bin, img):
        """
        Metodo para la imagen no hay qeu procesarla completa porque solo tiene una region
        marcada con lineas tenemos qque recortarla
        
        @param img_bin: imagen binarizada.
        @param img: imagen normal.
        
        @return: imgBinCrop: Es la imagen binarizada recortada a la region que queremos.
        @return: imgCrop: Es la imagen original recortada a la region que queremos.
        """
        img_bin_crop = img_bin[0:750, 500:1500]
        img_crop = img[0:750, 500:1500]
        return img_bin_crop, img_crop


    @classmethod
    def reducir_grosor(self, img_bin_crop):
        """
        Metodo para:
        Esta funcion en lo que consiste es qeu entrando la imagen binarizada 
        reduzcamos el tamano de las lineas para que la deteccion tenga menos
        incertidumbre y sea mas rapida la deteccion de lineas.
        
        @param img_bin_crop: Es la imagen binarizada recortada a la region que queremos.

        @return: sinRuido: Es la imagen una vez corregida la binarizacion apra que las 
                lineas sean mas delgadas.
        """
        sin_ruido = skeletonize(img_bin_crop)
        return sin_ruido

    
    @classmethod
    def pro_hough(self, threshold, line_length, line_gap, sin_ruido):
        """
        Este metodo lo que va a hacer es calcular los segmentos(ORIGEN"Ox,Oy" Y FIN "FX,FY") 
        de cada una de las lineas que se han detectado en la imagen binarizada atraves de la 
        funcion probabilistica de la transformada de hough.
        Entrada:
        
        @param threshold: Densidad de lineas. 
        @param line_length: Distancia minima de las lineas que calcula.
        @param line_gap: Distancia a la que si estan los extremos de las rectas las una.
        @param sin_ruido: Es la imagen una vez corregida la binarizacion apra que las 
            lineas sean mas delgadas.
            
        @return: lines: Lista de todos los segmentos que ha detectado.
        """
        lines = probabilistic_hough_line(sin_ruido, threshold, line_length, line_gap)
        return lines
    
    @classmethod
    def binarizar_para_cuadrado(self,img):
        """
        Este metodo va a consistir en que a partir de la imagen que hemos abierto
        nos la va a binarizar resaltando unicamente los bordes del cuadrado donde
        detectar las lineas 
        @param img: imagen que hemos leido.
        
        @return: imagen binarizada resaltando bordes. 
        """
        img_hsv=rgb2hsv(img)
        grises = rgb2grey(img_hsv)
        ii=0
        jj=0
        for i in grises:
            for j in i:
                if j>0 and j<0.0025:
                    grises[ii][jj]=1
                else:
                    grises[ii][jj]=0
                jj=jj+1
            jj=0
            ii=ii+1
        return grises
    
    @classmethod
    def obtener_max_y_min(self,lines):
        """
        En este metodo vamos a calcular las xmax, xmin ymax e ymin de neustro cuadrado
        es decir los 4 vertices que componen el cuadrado
        @param lines: lista de puntos que forman las lineas del cuadrado.
        @return: 4 vertices del cuadrado.
        """
        puntos_x,puntos_y=set(),set()
        for i in lines:
            puntos_x.add(i[0][0])
            puntos_x.add(i[1][0])    
            puntos_y.add(i[0][1])
            puntos_y.add(i[1][1])
        if len(lines)==0:
            return 0,0,0,0
        return max(puntos_x),min(puntos_x),max(puntos_y),min(puntos_y)
    
    @classmethod
    def pertenece_o_no(self,x,y,x_min,x_max,y_min,y_max):
        """
        En este metodo dado un punto y los 4 vertices del cuadrado comprobaremos
        si las coordenadas del punto pertenecen al area pintable.
        @param x: coordenada x del punto a comprobar.
        @param y: coordenada y del punto a comprobar.
        @param x_min: coordenada x minima del cuadrado
        @param x_max: coordenada x macima del cuadrado.
        @param y_min: coodenada y minima del cuadrado.
        @param y_max: coordenada y maxima del cuadrado.
        @return: true/false dependiendo si pertenece a esa region o no.     
        """
        if x == None and y==None:
            return False
        if (x_min< x < x_max) and  (y_min< y < y_max):
            return True
        else:
            return False

    @classmethod
    def obtener_numeros(self,img):
        """
        Metodo que se va a encargar de obtener los numero que componen
        la referencia.
        @param img: imagen de la que obtener la referencia. 
        @return: devolvemos el valor leido de la imagen que sera la referencia.
        """
        img_crop = img[860:950,520:645]
        img_hsv=rgb2hsv(img_crop)
        distance_red = rgb2grey(1 - np.abs(img_hsv - (1, 1, 0)))

        #binarizar
        ii=0
        jj=0

        for i in distance_red:
            for j in i:
                if round(j,2)>0 and round(j,2)<0.09:
                    distance_red[ii][jj]=1
                else:
                    distance_red[ii][jj]=0
                jj=jj+1
            jj=0
            ii=ii+1

        temp = tempfile.mkdtemp()
        #guardar en temp
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            io.imsave(temp+self.dic.pro_img,distance_red)

        act=os.getcwd()
        tes=act+self.dic.pro_tessera
        dir_img=temp+self.dic.pro_img_tesse
        dir_salida=temp+self.dic.pro_sal
        opt=self.dic.pro_batch

        #ejecutar tesseract
        os.system(tes+dir_img+dir_salida+opt)
        #obtener resultado de la ejecucion.
        f = open(temp+self.dic.pro_sal_txt)
        g=f.read()
        f.close()
        #borrar temporal.
        shutil.rmtree(temp) 
        return g.replace('3','0').replace('\n','').replace('8','0')
    @classmethod
    def pixel_rgb_2_lab(self,pixel):
        """
        Esta funcion se encargara de pasar un pixel de rgb a lab.
        @param pixel: pixel que pasar a espacio de color lab.
        @return: pixel en espacio de color lab. 
        """
        r,g,b = pixel        
        return rgb2lab([[[r/255,g/255,b/255]]])[0][0]
    
    @classmethod
    def pixelrgb_2_hsv(self,pixel):
        """
        Esta funcion se encargara de pasar un pixel de rgb a hsv.
        @param pixel: pixel que pasar a espacio de color hsv.
        @return: pixel en espacio de color hsv. 
        """
        r,g,b = pixel
        return rgb2hsv([[[r/255,g/255,b/255]]])[0][0]

In [8]:

class Diccionario():
    """ 
    Clase que contiene los string necesarios para inicializar los datos..
        
    @author: Ismael Tobar Garcia
    @version: 1.0    
    """
    def __init__(self):
        #Estadisticas      
        self.v='v'
        self.h='h'
        self.md='md'
        self.dm='dm'
        self.totales='totales'
        self.proyec="proyect"
        self.idioma="Ingles"
        self.idioma_esp="Español"
        self.idioma_selec="Idioma"
        self.info_msg="Reinicializar para aplicar cambios \n La aplicacion se va a cerrar"
        self.warnn="AVISO"
        
        self.docu3="docu3"
        self.docu4="docu4"
        self.docu5="docu5"
        #pestannas nombres de guardar
        self.docu="docu0"
        self.docu1="docu1"
        self.docu2="docu2"
   
        
        #Configuracion a xml
        self.pintada='/Pintada.jpg'
        self.pro_xml="/Proyecto.xml"
        self.utf='UTF-8'
        self.repeti="repeti"
        self.long="long"
        self.direccion="direccion"
        self.path="path"
        self.tex='tex'
        self.tab='/Tabla.tex'
        self.csv='csv'
        self.sal_estad='/Salida_Estadisticas.csv' 
        self.sal_lin='/Salida_Lineas.csv' 
        self.jpg1='jpg1'
        self.origi='/Original.jpg'
        self.jpg2='jpg2'
      
        #datosTo csv
        self.cabecera_csv_lineas=['linea', 'angulo', 'tamano', 'tipo']
        self.cabecera_csv_estad=['tipo', 'numero', 'mediaLon', 'desviacionTip']
        self.xmin="xmin"
        self.ymin="ymin"
        

        #Informe
        self.jin_alm='/#{'
        self.jin_porc='%%'
        self.jin_alm_po='%#'
        self.infor_plan='proyecto/codigo/informes/jinja-test.tex'
        self.tab_in='Tabla.tex'
        self.jin_blo='/BLOCK{'
        self.jin_o_q='}'
        self.jin_va=r'\VAR{' 
    
        
        
        #ProcesadoImagen
        self.jpg="JPEG"
        self.pro_img="/imagen.png"
        self.pro_img_tesse="/imagen.png "
        self.pro_tessera="/tesseract/tesseract.exe "
        self.pro_sal="/salida "
        self.pro_batch="nobatch digits "
        self.pro_sal_txt='/salida.txt'
        
        
        #MediadorPestannas
        self.md_pe_automati="calcula Auto"
        self.md_pe_fijar="Fijar cuadrado"
        self.md_pe_corregir="Corregir lineas"
        self.md_pe_tablas="Tabla"
        self.md_pe_lineas_pintadas="Detectar Pintadas"
        self.md_pe_automatico="Automatico"
        
        self.md_pe_lin_pin=""
        self.md_pe_calc='Calcular Lineas'
        self.md_pe_param = "Guardar parametros" 
        self.md_pe_repe = "Repeticiones:" 
        self.md_pe_long_min = "Ignorar menores de:" 
        self.md_pe_direccion = "Direccion de imagen" 
        self.sel_col="Selecionar color"  
        self.md_pe_col="Color: "
        self.md_pe_no_sel="No seleccionado"
        
        self.md_pe_corre="Corregir lineas"
        self.md_pe_auto="Automatico"
        self.md_pe_corre='Corregir Lineas'
        self.md_pe_anadir_p='Anadir punto'
        self.md_pe_anadir_seg='Anadir segmentos'
        self.md_pe_borrar='Borrar seleccionado'
        self.md_pe_guardar='Guardar tabla'
        self.md_pe_limpiar='Limpiar tabla'
        self.md_pe_p1="P_1:"
        self.md_pe_p2="P_2:"
        self.md_pe_cero="0"
        self.md_pe_cero="0"
        self.md_pe_cero="0"
        self.md_pe_cero="0"
        self.md_pe_cabe_tab=['P1X', 'P1Y', 'P2X', 'P2Y']
        self.md_pe_amarillo='yellow'
        self.md_pe_color_bl='color: black'
        self.md_pe_color_red='color: Red'
        self.md_pe_but_press='button_press_event'
        self.md_pe_ok="OK"
        self.md_pe_msg_sob="¿Esta seguro de sobreescribir?"
        self.md_pe_msg_inf="La carpeta ya existe"
        self.md_pe_msg_avi="Aviso"
        self.md_pe_open="openFolder"
        self.md_pe_proy='/Proyecto'
        self.md_pe_war="Warning:"
        self.md_pe_ori='/Proyecto/Original.jpg'
        self.md_pe_pin='/Proyecto/Pintada.jpg'
        self.md_pe_pro='/Proyecto/Proyecto.xml'
        self.md_pe_est='/Proyecto/Salida_Estadisticas.csv'
        self.md_pe_err_st="Falla porque esta abierto el archivo Salida_Estadisticas"
        self.md_pe_lin='/Proyecto/Salida_Lineas.csv'
        self.md_pe_err_lin="Falla porque esta abierto el archivo Salida_Lineas"
        self.md_pe_tab='/Proyecto/Tabla.tex'
        self.md_pe_err_tab="Falla porque esta abierto el archivo Tabla"
        self.md_pe_msg_gur="No se han guardado los cambios."
        self.md_pe_direc=["Derecha","Izquierda"]
        self.md_pe_puntos_cuadrado=["-","-","-","-"]
        self.md_pe_orien="Orientacion del diente"
        #mediadorVentana
        self.md_v_ori='nearest'
        self.md_v_up='upper'
        self.md_v_color='b'
        
        #VentanaInicio
        self.tradu="ESP"
        self.traduc="traduc"
        self.ini_p_tradu="idiom"
        
        self.ini_log='logger.log'
        self.ini_nuevo="&Nuevo proyecto"
        self.ini_o_nuevo="Ctrl+O"
        self.ini_p_abrir='Abrir imagen'
        self.ini_abrir_pro="&Abrir proyecto"
        self.ini_o_abrir_pro="Ctrl+A"
        self.ini_p_abrir_pro='Abrir Proyecto'
        self.ini_salir="&Salir"
        self.ini_o_salir="Ctrl+E"
        self.ini_p_salir='Salir'
        self.ini_guardar="&Guardar proyecto"
        self.ini_o_guardar="Ctrl+G"
        self.ini_p_guardar='Guardar csv y .tex'
        self.ini_acerca="&Acerca de"
        self.ini_o_ayuda='Ayuda'
        self.ini_ayuda="&Ayuda"
        self.ini_archivo='&Archivo'
        self.nombre_api='DietaPorDientes'
        
        self.ini_msg="Cargar imagen para iniciar"
        self.ini_color='color: black'
        self.ini_time="Times"        
        self.ini_msg_acerca="Autores: \n\tIsmael Tobar García \n\tAlvar Gonzalez Arnaiz\n\tJose Francisco Diez Pastor\nVersion: \n\t1.0 "
        self.ini_acercade="Acerca de"  
        self.ini_p_abri='Abrir imagen'
        self.ini_p_dir='c:/'
        self.ini_p_opt="Image files (*.jpg )"
        self.ini_p_war="Warning:"
        self.ini_p_err="Error:"
        self.ini_p_ok="OK"
        self.ini_p_cambios="Se han detectado cambios desea guardar"
        self.ini_p_aviso="Aviso"
        self.ini_p_cargar="Cargar Proyecto"
        self.ini_p_war_amp="Warning: fichero csv no existe"
        
        #Pintar rectangulo:
        self.detect_event_press='button_press_event'
        self.detect_event_release='button_release_event'
        self.detect_event_motion='motion_notify_event'
        
        self.md_pe_lin_pin="Detectores"
        self.md_pe_calc='Calcular Lineas'
        self.md_pe_param = "Guardar parametros" 
        self.md_pe_repe = "Repeticiones:" 

        self.md_pe_long_min = "Ignorar menores de:" 
        self.md_pe_direccion = "Direccion de imagen" 
        self.sel_col="Selecionar color"  
        self.md_pe_col="Color: "
        self.md_pe_no_sel="No seleccionado"
        self.md_pe_direc=["Derecha","Izquierda"]

        self.md_pe_corre="Corregir lineas"
        self.md_pe_auto="Automatico"
        self.md_pe_corre='Corregir Lineas'
        self.md_pe_anadir_p='Anadir punto'
        self.md_pe_anadir_seg='Anadir segmentos'
        self.md_pe_borrar='Borrar seleccionado'
        self.md_pe_guardar='Guardar tabla'
        self.md_pe_limpiar='Limpiar tabla'
        
        self.md_pe_msg_sob="¿Esta seguro de sobreescribir?"
        self.md_pe_msg_inf="La carpeta ya existe"
        self.md_pe_msg_avi="Aviso"
        self.md_pe_open="openFolder"
        self.md_pe_war="Warning:"
        self.tradu="ESP"
        self.md_pe_msg_gur="No se han guardado los cambios."
        
        #VentanaInicio
        self.ini_nuevo="&Nuevo proyecto"
        self.ini_p_abrir='Abrir imagen'
        self.ini_abrir_pro="&Abrir proyecto"
        self.ini_p_abrir_pro='Abrir Proyecto'
        self.ini_salir="&Salir"
        self.ini_p_salir='Salir'
        self.ini_guardar="&Guardar proyecto"
        self.ini_p_guardar='Guardar csv y .tex'
        self.ini_acerca="&Acerca de"
        self.ini_o_ayuda='Ayuda'
        self.ini_ayuda="&Ayuda"
        self.ini_archivo='&Archivo'
        self.nombre_api='DietaPorDientes'
        
        self.ini_msg="Cargar imagen"
        self.ini_msg2="o "
        self.ini_msg3="cargar proyecto "
        self.ini_msg4="para iniciar. "
        
        self.ini_time="Times"        
        self.ini_msg_acerca="Autores: \n\tIsmael Tobar García \n\tAlvar Gonzalez Arnaiz\n\tJose Francisco Diez Pastor\nVersion: \n\t1.0 "
        self.ini_acercade="Acerca de"  
        self.ini_p_abri='Abrir imagen'
        self.ini_p_war="Warning:"
        self.ini_p_err="Error:"
        self.ini_p_ok="OK"
        self.ini_p_cambios="Se han detectado cambios desea guardar"
        self.ini_p_aviso="Aviso"
        self.ini_p_cargar="Cargar Proyecto"
